In [1]:
from dotenv import load_dotenv
load_dotenv()


import os 
import pinecone
print(pinecone.__version__)
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

import sys
print(sys.executable)



5.0.1
c:\Users\kvive_qz6f7mf\anaconda3\envs\rag\python.exe


In [2]:
load_dotenv('.env.local')
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")

)

In [10]:
import json

# Load the JSON data
data = json.load(open("reviews.json"))

# Print the loaded data to see its structure
print(type(data))
print(data)


<class 'dict'>
{'reviews': [{'Professor': 'Dr. Jane Smith', 'subject': 'Physics', 'stars': 5, 'review': 'Dr. Smith is an excellent professor. She explains complex topics in a very clear and understandable way. Her lectures are engaging, and she is always willing to help students.'}, {'Professor': 'Dr. John Doe', 'subject': 'Mathematics', 'stars': 4, 'review': "Dr. Doe is very knowledgeable, but his lectures can be a bit fast-paced. It's important to keep up with the material. However, he is always available during office hours."}, {'Professor': 'Dr. Emily Johnson', 'subject': 'Chemistry', 'stars': 3, 'review': 'Dr. Johnson is a decent professor, but her lectures can be a bit monotonous. She knows the subject well, but sometimes struggles to convey the material effectively.'}, {'Professor': 'Dr. Mark Williams', 'subject': 'History', 'stars': 2, 'review': "Dr. Williams is a tough grader and his lectures are often dry. He expects a lot from students, but doesn't provide much support outsi

In [13]:
processed_data = []

client = OpenAI()

for review in data['reviews']:
    response=client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"

    )

    embedding = response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review["Professor"],
        "metadata":{
            "review":review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [14]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [15]:

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}